# Accelerating End-to-End Data Science Workflows # 


## 04 - การทำงานร่วมกันของ GPU PyData Ecosystem ##

**สารบัญ**

โน้ตบุ๊กนี้จะนำเสนอตัวอย่างวิธีการใช้ cuDF และ CuPy ร่วมกันเพื่อใช้ประโยชน์จากฟังก์ชันการทำงานของ CuPy array (เช่น การดำเนินการพีชคณิตเชิงเส้นขั้นสูง) โน้ตบุ๊กนี้ครอบคลุมส่วนต่างๆ ดังนี้:

1.  [NumPy, SciPy, และ CuPy](#NumPy,-SciPy,-and-CuPy)
    * [cuDF vs. CuPy](#cuDF-vs.-CuPy)
2.  [การทำงานกับ CuPy](#Working-with-CuPy)
3.  [ตัวแปลงพิกัดกริด (Grid Converter)](#Grid-Converter)
    * [ตัวแปลงพิกัดละติจูด/ลองจิจูดเป็น OSGB Grid ด้วย NumPy](#Lat/Long-to-OSGB-Grid-Converter-with-NumPy)
    * [ตัวแปลงพิกัดละติจูด/ลองจิจูดเป็น OSGB Grid ด้วย CuPy](#Lat/Long-to-OSGB-Grid-Converter-with-CuPy)
    * [แบบฝึกหัดที่ 1 - การเพิ่มคอลัมน์พิกัดกริดลงใน DataFrame](#Exercise-#1---Adding-Grid-Coordinate-Columns-to-DataFrame)
4.  [การจัดทำดัชนีอาร์เรย์แบบบูลีน (Boolean Array Indexing)](#Boolean-Array-Indexing)



## NumPy, SciPy และ CuPy ##

ตามคู่มือผู้ใช้ของตนเอง [NumPy](https://numpy.org/doc/stable/user/whatisnumpy.html) เป็นแพ็คเกจพื้นฐานสำหรับการคำนวณทางวิทยาศาสตร์ใน Python เป็นไลบรารี Python ที่มี **วัตถุอาร์เรย์หลายมิติ** วัตถุที่ได้มาต่างๆ (เช่น masked arrays และ matrices) และชุดของฟังก์ชันสำหรับการดำเนินการที่รวดเร็วบนอาร์เรย์ การดำเนินการเหล่านี้รวมถึงทางคณิตศาสตร์ ตรรกะ การจัดการรูปร่าง การเรียงลำดับ การเลือก I/O การแปลงฟูริเยร์แบบไม่ต่อเนื่อง พีชคณิตเชิงเส้นพื้นฐาน การดำเนินการทางสถิติพื้นฐาน การจำลองแบบสุ่ม และอื่นๆ อีกมากมาย ในขณะที่ NumPy มุ่งเน้นไปที่อาร์เรย์ การดำเนินการทางคณิตศาสตร์ และพีชคณิตเชิงเส้นพื้นฐาน [SciPy](https://docs.scipy.org/doc/scipy-1.8.1/tutorial/general.html) สร้างขึ้นบนรากฐานนี้เพื่อมอบฟังก์ชันการทำงานเพิ่มเติม โดยเฉพาะอย่างยิ่งในโดเมนของการคำนวณทางวิทยาศาสตร์และการปรับให้เหมาะสม

ในทางกลับกัน [CuPy](https://cupy.dev/) เป็นไลบรารีอาร์เรย์โอเพนซอร์สสำหรับการคำนวณที่เร่งความเร็วด้วย GPU ด้วย Python CuPy สามารถมองได้ว่าเป็นคู่หูที่เร่งความเร็วด้วย GPU ของ NumPy โดยนำเสนอฟังก์ชันการทำงานและ API ที่คล้ายกัน พร้อมด้วยประโยชน์เพิ่มเติมของการเร่งความเร็วด้วย GPU สำหรับภาระงานที่เข้ากันได้ ในขณะที่ NumPy ทำงานบนหน่วยความจำ CPU CuPy จะทำงานกับหน่วยความจำ GPU เป็นหลัก โดยใช้ประโยชน์จาก GPU ที่เปิดใช้งาน CUDA สำหรับการคำนวณ อินเทอร์เฟซของ CuPy เข้ากันได้สูงกับ NumPy และ SciPy ในกรณีส่วนใหญ่สามารถใช้แทนกันได้ สิ่งที่เราต้องทำคือเพียงแค่แทนที่ `numpy` และ `scipy` ด้วย `cupy` และ `cupyx.scipy` ในโค้ด Python สิ่งนี้ทำให้ผู้ใช้ที่คุ้นเคยกับ NumPy สามารถเปลี่ยนไปใช้การคำนวณที่เร่งความเร็วด้วย GPU ได้ง่ายขึ้น

CuPy ได้รับการออกแบบมาให้ทำงานร่วมกับไลบรารีที่เร่งความเร็วด้วย GPU อื่นๆ ในระบบนิเวศ RAPIDS ได้อย่างราบรื่น คล้ายกับที่ NumPy ทำงานกับ pandas และไลบรารีที่ใช้ CPU อื่นๆ การเก็บข้อมูลไว้บน GPU ตลอดเวิร์กโฟลว์ช่วยให้เราสามารถลดค่าใช้จ่ายในการถ่ายโอนข้อมูลระหว่างหน่วยความจำ CPU และ GPU



### cuDF vs. CuPy ###

จนถึงตอนนี้ DataFrame ที่เราใช้งานมานั้นได้จัดเก็บข้อมูลในรูปแบบตารางที่มีโครงสร้าง ซึ่งมีประโยชน์เมื่อเราต้องการดำเนินการที่เหมือนกับ DataFrame เช่น การจัดกลุ่ม การรวมข้อมูล การกรอง และการรวมข้อมูล อย่างไรก็ตาม อาจมีกรณีการใช้งานที่จำเป็นต้องทำงานกับอาร์เรย์หลายมิติหรือเมทริกซ์ เช่น การดำเนินการทางพีชคณิตเชิงเส้นหรืองานคำนวณทางวิทยาศาสตร์ สำหรับกรณีเหล่านี้ เราจะต้องการใช้ไลบรารีที่ออกแบบมาสำหรับการทำงานเหล่านี้โดยเฉพาะ เช่น NumPy, SciPy หรือ CuPy กล่าวอีกนัยหนึ่งคือ ใช้ cuDF เมื่อทำงานกับการจัดการข้อมูลระดับสูง (นามธรรมน้อยกว่า) และใช้ CuPy เมื่อทำการดำเนินการตัวเลขระดับต่ำกับอาร์เรย์หลายมิติ

ในทางปฏิบัติ นักวิทยาศาสตร์ข้อมูลส่วนใหญ่ทำงานกับทั้งสองไลบรารี เนื่องจากเวิร์กโฟลว์ส่วนใหญ่ของพวกเขาเกี่ยวข้องกับการดำเนินการ DataFrame และการคำนวณแบบอาร์เรย์ ตัวอย่างเช่น เราอาจใช้ cuDF สำหรับการโหลดและประมวลผลข้อมูล จากนั้นแปลงเป็นอาร์เรย์ CuPy สำหรับการคำนวณตัวเลขเฉพาะ และแปลงกลับเป็น cuDF เพื่อการวิเคราะห์เพิ่มเติมหรือส่งออก cuDF และ CuPy ได้รับการออกแบบมาให้ทำงานร่วมกันได้ ทำให้เราสามารถแปลงระหว่าง cuDF DataFrames/Series และอาร์เรย์ CuPy ได้อย่างง่ายดายโดยยังคงข้อมูลไว้บน GPU สิ่งนี้ช่วยให้เราสามารถสร้างเวิร์กโฟลว์ที่มีประสิทธิภาพซึ่งใช้ประโยชน์จากจุดแข็งของทั้งสองไลบรารี


## การทำงานกับ CuPy ##

มีหลายวิธีในการใช้งาน CuPy จาก cuDF คุณสมบัติ `DataFrame.values` จะส่งคืน CuPy representation ของ data frame นอกจากนี้ เรายังสามารถแปลงผ่าน CUDA array interface โดยใช้ `DataFrame.to_cupy()` นอกจากนี้ เรายังสามารถส่ง Series ไปยังฟังก์ชัน `cupy.asarray()` ได้ เนื่องจาก cuDF Series แสดง CUDA array interface ซึ่งเป็นแนวทางที่เร็วที่สุด

ด้านล่างนี้ เราจะสาธิต **การบวกแบบแถวต่อแถว (row-wise sum)** บน DataFrame การสนับสนุนการดำเนินการแบบแถวต่อแถวของ cuDF ยังไม่สมบูรณ์นัก ดังนั้นเราจะต้องทำการ transpose DataFrame หรือเขียน UDF และคำนวณผลรวมข้ามแต่ละแถวอย่างชัดเจน การ transpose อาจทำให้มีหลายแสนคอลัมน์ (ซึ่ง cuDF จะไม่ทำงานได้ดี) ขึ้นอยู่กับรูปร่างข้อมูลของเรา และการเขียน UDF อาจใช้เวลานาน การใช้ประโยชน์จากการทำงานร่วมกันของ GPU PyData ecosystem ทำให้การดำเนินการนี้ง่ายมาก

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# DO NOT CHANGE THIS CELL
# import libraries
import cudf
import time

In [ ]:
# DO NOT CHANGE THIS CELL
num_ele = 1000000

df = cudf.DataFrame(
    {
        "a": range(num_ele),
        "b": range(10, num_ele + 10),
        "c": range(100, num_ele + 100),
        "d": range(1000, num_ele + 1000)
    }
)

# preview
df.head()

In [ ]:
# DO NOT CHANGE THIS CELL
start=time.time()
display(df.sum(axis=1))
time.time()-start

การดำเนินการเดียวกันทำงานได้เร็วกว่าด้วย CuPy

In [ ]:
# DO NOT CHANGE THIS CELL
arr=df.values

start=time.time()
# alternative approach
# arr=df.to_cupy()

display(arr.sum(axis=1))
time.time()-start



เมื่อใช้ cuDF pandas เราสามารถใช้คุณสมบัติ `.values` รวมถึงฟังก์ชัน `cupy.asarray()` ได้ด้วย

**หมายเหตุ**: เราสามารถใช้เมธอด `.to_numpy()` เพื่อแปลง cuDF DataFrames หรือ Series เป็น NumPy arrays ได้

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# DO NOT CHANGE THIS CELL
%load_ext cudf.pandas
import pandas as pd

import numpy as np
import cupy as cp
import time

In [ ]:
# DO NOT CHANGE THIS CELL
num_ele = 1000000

df = pd.DataFrame(
    {
        "a": range(num_ele),
        "b": range(10, num_ele + 10),
        "c": range(100, num_ele + 100),
        "d": range(1000, num_ele + 1000)
    }
)

# preview
df.head()

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
arr=df.values
# alternative approach
# arr=cp.asarray(df)

start=time.time()

display(arr.sum(axis=1))

time.time()-start


เช่นเดียวกับที่เราสามารถทำได้กับ NumPy และ pandas เราสามารถเชื่อมโยง cuDF และ CuPy เข้าด้วยกันในเวิร์กโฟลว์เดียวกัน โดยยังคงข้อมูลทั้งหมดไว้บน GPU เราสามารถย้ายไปมาระหว่างโครงสร้างข้อมูลในระบบนิเวศนี้ได้อย่างราบรื่น ทำให้เรามีความยืดหยุ่นอย่างมากโดยไม่ลดความเร็ว หากเรากำลังทำงานกับ RAPIDS cuDF แต่ต้องการฟังก์ชันที่เน้นพีชคณิตเชิงเส้นมากขึ้นซึ่งมีอยู่ใน CuPy เราสามารถใช้ประโยชน์จากการทำงานร่วมกันของ GPU PyData ecosystem เพื่อใช้ฟังก์ชันนั้นได้

ในการแปลง CuPy array เป็น cuDF DataFrame หรือ Series เราสามารถใช้คอนสตรัคเตอร์ของแต่ละรายการได้

In [ ]:
# DO NOT CHANGE THIS CELL
df['sum']=arr.sum(axis=1)

df.head()


## ตัวแปลงพิกัดกริด (Grid Converter) ##

ข้อมูลส่วนใหญ่ของเรามีพิกัดละติจูดและลองจิจูด แต่สำหรับบางงานที่เกี่ยวข้องกับระยะทาง เช่น การระบุกลุ่มผู้ติดเชื้อที่มีความหนาแน่นทางภูมิศาสตร์สูง การหาโรงพยาบาลหรือคลินิกที่ใกล้ที่สุดจากบุคคลที่กำหนด จะสะดวกกว่าถ้ามีพิกัดกริดคาร์ทีเซียนแทน โดยใช้การฉายแผนที่เฉพาะภูมิภาค (ในกรณีนี้คือ [Ordnance Survey Great Britain 1936](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid)) เราสามารถคำนวณระยะทางในท้องถิ่นได้อย่างมีประสิทธิภาพและมีความแม่นยำที่ดี

In [ ]:
# DO NOT CHANGE THIS CELL
dtype_dict={
    'age': 'int8', 
    'sex': 'category', 
    'county': 'category', 
    'lat': 'float32', 
    'long': 'float32', 
    'name': 'category'
}
        
df=pd.read_csv('./data/uk_pop.csv', dtype=dtype_dict)
df.head()

### ตัวแปลงพิกัดละติจูด/ลองจิจูดเป็น OSGB Grid ด้วย NumPy ###

ในการแปลงพิกัด เราจะสร้างฟังก์ชัน `latlong2osgbgrid` ซึ่งรับพิกัดละติจูด/ลองจิจูด และแปลงเป็น [พิกัด OSGB36](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid): ค่า "northing" และ "easting" ซึ่งแสดงถึงระยะทางพิกัดคาร์ทีเซียนของจุดจากมุมตะวันตกเฉียงใต้ของกริด

ด้านล่างนี้คือ `latlong2osgbgrid` ซึ่งอาศัย NumPy เป็นหลัก:

In [ ]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (north)
    long: longitude coordinate (east)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * np.pi/180
        long = long * np.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000                # northing of true origin
    E0 = 400000                 # easting of true origin
    F0 = .9996012717            # scale factor on central meridian
    phi0 = 49 * np.pi / 180     # latitude of true origin
    lambda0 = -2 * np.pi / 180  # longitude of true origin and central meridian
    
    sinlat = np.sin(lat)
    coslat = np.cos(lat)
    tanlat = np.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * np.sin(latdiff) * np.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * np.sin(2*(latdiff)) * np.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * np.sin(3*(latdiff)) * np.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - np.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

### ตัวแปลง Lat/Long เป็น OSGB Grid ด้วย CuPy ###

ในฟังก์ชัน `latlong2osgbgrid_cupy` ที่จะตามมา เราเพียงแค่สลับ `cp` แทน `np` ในขณะที่ CuPy รองรับงานที่เร่งความเร็วด้วย GPU ที่ทรงพลังหลากหลายประเภท เทคนิคที่เรียบง่ายนี้ในการสามารถสลับการเรียกใช้ CuPy แทนการเรียกใช้ NumPy ทำให้เป็นเครื่องมือที่มีประสิทธิภาพอย่างไม่น่าเชื่อที่เรามีพร้อมใช้งาน

In [ ]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (north)
    long: longitude coordinate (east)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000                 # northing of true origin
    E0 = 400000                  # easting of true origin
    F0 = .9996012717             # scale factor on central meridian
    phi0 = 49 * cp.pi / 180      # latitude of true origin
    lambda0 = -2 * cp.pi / 180   # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)



ด้านล่างนี้ เราจะส่งพิกัดละติจูด/ลองจิจูดไปยังตัวแปลง ซึ่งจะส่งค่าเหนือและตะวันออกภายใน OSGB grid กลับมาให้

In [ ]:
%%time
# DO NOT CHANGE THIS CELL
numpy_lat = np.asarray(df['lat'])
numpy_long = np.asarray(df['long'])

In [ ]:
%%time
# DO NOT CHANGE THIS CELL
n_numpy_array, e_numpy_array = latlong2osgbgrid(numpy_lat, numpy_long)

### แบบฝึกหัดที่ 1 - การเพิ่มคอลัมน์พิกัดกริดลงใน DataFrame ###

ตอนนี้เราจะใช้ `latlong2osgbgrid_cupy` เพื่อเพิ่มคอลัมน์ `northing` และ `easting` ลงใน `df` เราจะเริ่มต้นด้วยการแปลงสองคอลัมน์ที่เราต้องการ คือ `lat` และ `long` ให้เป็น CuPy arrays ด้วยฟังก์ชัน `cp.asarray()` เนื่องจาก cuDF และ CuPy เชื่อมต่อกันโดยตรงผ่าน `__cuda_array_interface__` การแปลงจึงเกิดขึ้นได้ในหน่วยนาโนวินาที

**คำแนะนำ**:
* รันเซลล์ด้านล่างเพื่อสร้าง CuPy arrays สำหรับคอลัมน์ `lat` และ `long`
* แก้ไขเฉพาะส่วน `<FIXME>` และรันเซลล์ด้านล่างเพื่อใช้ `latlong2osgbgrid_cupy` กับ `cupy_lat` และ `cupy_long` จากนั้นเพิ่มเป็นคอลัมน์ `northing` และ `easting` โดยมี dtype เป็น `float32`

In [ ]:
%%time
# DO NOT CHANGE THIS CELL
cupy_lat = cp.asarray(df['lat'])
cupy_long = cp.asarray(df['long'])

In [ ]:
%%time
n_cupy_array, e_cupy_array = <<<<FIXME>>>>
df['northing'] = <<<<FIXME>>>>
df['easting'] = <<<<FIXME>>>>
print(df.dtypes)
df.head()

Click ... for solution. 

## การจัดทำดัชนีอาร์เรย์แบบบูลีน (Boolean Array Indexing) ##

ด้านล่างนี้ เราจะใช้ `np.logical_and` สำหรับการเลือกค่าบูลีนแบบทีละองค์ประกอบ

In [ ]:
# DO NOT CHANGE THIS CELL
start=time.time()
display(df.loc[np.logical_and(df['name'].str.startswith('E'), df['name'].str.endswith('D'))].head())
print(f'Duration: {round(time.time()-start, 2)} seconds')

ด้านล่างนี้ เราจะใช้ CuPy สำหรับการเลือกแบบบูลีน


In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
start=time.time()
display(df.loc[cp.logical_and(df['name'].str.startswith('E'), df['name'].str.endswith('D'))].head())
print(f'Duration: {round(time.time()-start, 2)} seconds')

**หมายเหตุ**: อาร์เรย์สตริงยังไม่ได้ถูกนำมาใช้ใน CuPy

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [โน้ตบุ๊กถัดไป](1-05_grouping.ipynb) กันเลย